线性代数，起源于对线性方程组（system of linear equations）的解（solution）的研究。
而矩阵，最初是线性方程组的简便写法。

## 一、从线性方程组到矩阵

线性方程组

$\begin{cases}
    2x&-y&=0\\
    -x&+2y&=3
\end{cases}$，$(x, y)$ 可看作两条直线的交点。

可以把系数写成列向量，简写成下列形式

$\begin{bmatrix}
    2\\
    -1
\end{bmatrix}x+
\begin{bmatrix}
    -1\\
    2
\end{bmatrix}y=
\begin{bmatrix}
    0\\
    3
\end{bmatrix}$

显然，上述方程的左边，是两个**列向量的线性组合**。而 $x$ 和 $y$ 就是能使等式成立的组合系数。

在定义的矩阵的乘法后，可进一步简写成下列形式

$\begin{bmatrix}
    2&-1\\
    -1&2
\end{bmatrix}
\begin{bmatrix}
    x\\
    y
\end{bmatrix}=
\begin{bmatrix}
    0\\
    3
\end{bmatrix}$

抽象为 $Ax=b$，称作矩阵方程。

总结一下，就是可以从三个不同的角度来看待一个线性方程组：**矩阵方程、向量方程和线性方程组**。

## 二、方程组的解法

### 2.1 行简化算法

即对矩阵做若干**初等行变换**，将原矩阵变换成**行阶梯形式（row echelon form, rref）**。这一步也被称为高斯消元法（Gauss Elimination）

得到阶梯矩阵后，使用**矩阵回代法**可得矩阵的**行简化阶梯形式（row reduced echelon form, rref）**。这一步和上一步合在一起，称作行简化算法。

将上述的行简化算法应用到方程组的增广场矩阵上，将直接得出方程组的解。

In [147]:
# 行简化算法，输出为矩阵的 行简化阶梯形式
function rref(A::Matrix)
    A = Float64.(A)  # 统一转换为 Float64
    nr, nc = size(A)
    i = j = 1
    
    while i <= nr && j <= nc
        (m, mi) = findmax(abs.(A[i:end,j]))  # 选择一列中绝对值最大的元素作为主元（partial pivoting）
        mi = mi+i - 1  # 计算得 主元m 在 矩阵A 中的列索引
        
        if m <= eps(Float64)  # 说明 A[i:,j] 全为 0
            A[i:end, j] .= zero(Float64)
            j += 1  # 该列已经是最简了，直接跳到下一列
        else
            A[i, :], A[mi, :] = A[mi, :], A[i, :]  # 通过行交换，将主元移到主元位置上
            A[i, :] ./= A[i, j]  # 放缩主元所在行
            
            # 消除掉该列除主元外的所有元素
            for k = 1:nr
                if k != i
                    A[k, :] .-= A[i, :] * A[k, j] / A[i, j]
                end
            end
            
            i += 1
            j += 1
        end
    end
    A  # 返回结果
end

rref (generic function with 1 method)

In [142]:
# 有方程组 Ax = b
A = [3 -9 12 -9 6
     3 -7  8 -5 8
     0  3 -6  6 4]

b = [15
      9
     -5]

rref([A b])  # 计算增广场矩阵的行简化阶梯形式

3×6 Array{Float64,2}:
  1.0   0.0  -2.0   3.0  0.0  -24.0
  0.0   1.0  -2.0   2.0  0.0   -7.0
 -0.0  -0.0  -0.0  -0.0  1.0    4.0

## 三、解的存在性



线性方程组的解有三种可能：
1. 无解
2. 有唯一解
3. 有无穷解（有自有变量）

### 1. 无解

无解，从线性方程组的角度来说，就是存在互相矛盾的方程，比如
$$\begin{cases}
    x& + & y & = 0\\
    x& + & y & = 3
\end{cases}$$

从向量的线性组合的角度来说，就是 $向量b$ 不在 $A$ 的列向量所张成的向量空间之内。

从矩阵方程的角度来说，就是增广矩阵的阶梯形式包含如下形式的行（b 不为 0）
$$
\begin{bmatrix}
    0 & ... & 0 & b
\end{bmatrix}
$$
也就是说矩阵的增广列成了主元列，无解。

### 2. 有唯一解

方程组没有自由变量时，矩阵方程 $Ax=b$ 有唯一解。

这表示增广矩阵除增广列外的每一列都得是主元列，或者说增广矩阵的主元列数等于方程组的未知数个数。（后面我们会看到，这也是矩阵A可逆的充要条件）

### 3. 有无穷解

方程组存在自由变量时，方程组有无穷解。这是因为自由变量可以取任意值。

这时增广矩阵的主元列数少于方程组的未知数个数。